# Use Case: Find Competitor Filings Using the API

## Purpose

This workflow demonstrates the power of chaining multiple FinancialReports API endpoints to solve a common business problem: **finding recent annual reports from a company's direct competitors.**

This notebook will:
1.  **Start with a known company** (e.g., adidas AG, identified by its ISIN).
2.  **API Call #1:** Fetch the company's details to identify its industry classification (ISIC code).
3.  **API Call #2:** Find other companies operating in that same industry.
4.  **API Call #3 (in a loop):** For each competitor, retrieve their latest annual report filing.
5.  Display the final results in a clean, organized `pandas` DataFrame.

## 1. Setup

First, we import our required libraries, including the official FinancialReports Python SDK. We'll also set up our API client. 

**Important:** This notebook requires the `FR_API_KEY` environment variable to be set. See the root `README.md` for instructions.

In [ ]:
import os
import pandas as pd
from financial_reports_generated_client import Client
from financial_reports_generated_client.models import CompaniesListView
from financial_reports_generated_client.types import Response
from datetime import datetime

# --- API Client Setup ---
api_key = os.environ.get("FR_API_KEY")

if not api_key:
    print("ERROR: FR_API_KEY environment variable not set.")
else:
    # Base URL for the FinancialReports API
    base_url = "https://api.financialreports.eu"
    
    # Create an API client instance
    client = Client(base_url=base_url, headers={"X-API-Key": api_key}, timeout=30.0)
    print("API Client created successfully.")

## 2. The Workflow

We'll define a main function to encapsulate the entire logic. This makes the process clear and reusable.

In [ ]:
def find_competitor_annual_reports(target_isin: str, filing_type_code: str = "10-K"):
    """
    Finds the latest annual report for competitors of a given company.
    """
    if not api_key:
        print("API Key not configured. Aborting.")
        return None

    # --- Step 1: Find the target company and its industry ---
    print(f"Step 1: Finding details for company with ISIN {target_isin}...")
    try:
        # Use the 'full' view to get detailed industry data
        company_response = client.companies.companies_list(
            isin=target_isin,
            view=CompaniesListView.FULL
        )
        
        if not company_response.results or len(company_response.results) == 0:
            print(f"Could not find a company with ISIN {target_isin}.")
            return None
            
        target_company = company_response.results[0]
        company_name = target_company.name
        # The sub_industry is the most specific ISIC classification
        isic_code = target_company.sub_industry.code
        print(f"Found '{company_name}' in industry (ISIC): {isic_code}")
        
    except Exception as e:
        print(f"An error occurred during Step 1: {e}")
        return None

    # --- Step 2: Find other companies in the same industry ---
    print(f"\nStep 2: Searching for competitors in industry {isic_code}...")
    try:
        competitors_response = client.companies.companies_list(sub_industry=isic_code, page_size=10)
        # Filter out the original company from the competitor list
        competitors = [c for c in competitors_response.results if c.id != target_company.id]
        
        if not competitors:
            print(f"No other companies found in industry {isic_code}.")
            return None
        
        print(f"Found {len(competitors)} competitors, including: {[c.name for c in competitors[:3]]}...")
        
    except Exception as e:
        print(f"An error occurred during Step 2: {e}")
        return None

    # --- Step 3: Get the latest annual report for each competitor ---
    print(f"\nStep 3: Fetching latest '{filing_type_code}' for each competitor...")
    all_filings = []
    for competitor in competitors:
        try:
            latest_filing_response = client.filings.filings_list(
                company=competitor.id,
                type=filing_type_code,
                ordering="-release_datetime", # Get the most recent first
                page_size=1
            )
            
            if latest_filing_response.results:
                filing = latest_filing_response.results[0]
                all_filings.append({
                    'competitor_name': competitor.name,
                    'filing_title': filing.title,
                    'release_date': filing.release_datetime.date() if filing.release_datetime else None,
                    'filing_id': filing.id
                })
                print(f"  - Found filing for {competitor.name}")
        except Exception as e:
            print(f"  - Could not fetch filing for {competitor.name}: {e}")
    
    return pd.DataFrame(all_filings)


## 3. Run and Display Results

Now we can execute our workflow. We'll start with the ISIN for **adidas AG** (`DE000A1EWWW0`) and look for `10-K` or equivalent annual reports from its competitors.

In [ ]:
# ISIN for adidas AG
adidas_isin = "DE000A1EWWW0"

df_results = find_competitor_annual_reports(adidas_isin)

if df_results is not None:
    print("\n--- Competitor Annual Reports ---")
    display(df_results.sort_values(by='release_date', ascending=False).reset_index(drop=True))